In [ ]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
os.environ["GOOGLE_API_KEY"] = "" #1

In [24]:

# ---------- Imports ----------
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
# --- API KEYS ---
# GEMINI_API_KEY = ""
# GEMINI_API_KEY = os.getenv("GOOGLE_API_KEY") #1
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GEMINI_API_KEY,
)
res = model.invoke("Do you love me")
print(res)



ChatGoogleGenerativeAIError: Error calling model 'gemini-2.5-flash' (PERMISSION_DENIED): 403 PERMISSION_DENIED. {'error': {'code': 403, 'message': 'Your API key was reported as leaked. Please use another API key.', 'status': 'PERMISSION_DENIED'}}

In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_classic.chains import LLMChain
from langchain_classic.chains import SequentialChain
# from langchain.callbacks import get_openai_callback
import PyPDF2

In [8]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [9]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

NOTE
Dont add ```json  like in response

"""

In [10]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [11]:
quiz_chain=LLMChain(llm=model, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


C:\Users\Gourab Bistu\AppData\Local\Temp\ipykernel_26832\3091854445.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain=LLMChain(llm=model, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [12]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [ ]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)


In [14]:
review_chain=LLMChain(llm=model, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


In [15]:

generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
 output_variables=["quiz", "review"], verbose=True,)

In [16]:
file_path = "C:/Users/Gourab Bistu/Desktop/GENAI/project/Quize-Generator/data.txt"

In [17]:
file_path

'C:/Users/Gourab Bistu/Desktop/GENAI/project/Quize-Generator/data.txt'

In [18]:
with open(file_path,'r') as file:
    TEXT = file.read()

In [19]:
TEXT

'Machine learning is a branch of Artificial Intelligence that focuses on developing models and algorithms that let computers learn from data without being explicitly programmed for every task. In simple words, ML teaches systems to think and understand like humans by learning from the data.\n\nintroduction_to_machine_learning.webpintroduction_to_machine_learning.webp\n\nMachine Learning is mainly divided into three core types: Supervised, Unsupervised and Reinforcement Learning along with two additional types, Semi-Supervised and Self-Supervised Learning.\n\nSupervised Learning: Trains models on labeled data to predict or classify new, unseen data.\nUnsupervised Learning: Finds patterns or groups in unlabeled data, like clustering or dimensionality reduction.\nReinforcement Learning: Learns through trial and error to maximize rewards, ideal for decision-making tasks.\nNote: The following are not part of the original three core types of ML, but they have become increasingly important in

In [20]:
json.dumps(RESPONSE_JSON)


'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [21]:
NUMBER=5 
SUBJECT="machine Learning"
TONE="simple"

In [22]:
response=generate_evaluate_chain(
    {
        "text": TEXT,
        "number": NUMBER,
        "subject":SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    }
)
response

C:\Users\Gourab Bistu\AppData\Local\Temp\ipykernel_26832\3687884866.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  response=generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning is a branch of Artificial Intelligence that focuses on developing models and algorithms that let computers learn from data without being explicitly programmed for every task. In simple words, ML teaches systems to think and understand like humans by learning from the data.

introduction_to_machine_learning.webpintroduction_to_machine_learning.webp

Machine Learning is mainly divided into three core types: Supervised, Unsupervised and Reinforcement Learning along with two additional types, Semi-Supervised and Self-Supervised Learning.

Supervised Learning: Trains models on labeled data to predict or classify new, unseen data.
Unsupervised Learning: Finds patterns or groups in unlabeled data, like clustering or dimensionality reduction.
Reinforcement Learning: Learns through trial and error to maximize rewards, ideal for decision-making tasks.
Note: The following a

ChatGoogleGenerativeAIError: Error calling model 'gemini-2.5-flash' (PERMISSION_DENIED): 403 PERMISSION_DENIED. {'error': {'code': 403, 'message': 'Your API key was reported as leaked. Please use another API key.', 'status': 'PERMISSION_DENIED'}}

In [ ]:
quiz = response['quiz']
quiz

'{"1": {"mcq": "What are the three core types of Machine Learning mentioned in the text?", "options": {"a": "Supervised, Deep, and Neural Learning", "b": "Unsupervised, Semi-Supervised, and Self-Supervised Learning", "c": "Supervised, Unsupervised, and Reinforcement Learning", "d": "Classification, Regression, and Clustering"}, "correct": "c"}, "2": {"mcq": "In Supervised Learning, what is the main goal of a Regression task?", "options": {"a": "To predict discrete labels or categories", "b": "To find patterns in unlabeled data", "c": "To predict continuous numerical values", "d": "To learn through trial and error"}, "correct": "c"}, "3": {"mcq": "Which of these algorithms is typically used in Unsupervised Learning to group data points into clusters?", "options": {"a": "Linear Regression", "b": "K-Means clustering", "c": "Q-Learning", "d": "Decision Trees"}, "correct": "b"}, "4": {"mcq": "Which type of Machine Learning combines a small amount of labeled data with a large amount of unlab

In [ ]:
quiz
type(quiz)
python_dict = json.loads(quiz)

python_dict

In [ ]:
quiz_table_data = []
for key, value in python_dict.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [ ]:
quiz_table_data

[{'MCQ': 'What are the three core types of Machine Learning mentioned in the text?',
  'Choices': 'a: Supervised, Deep, and Neural Learning | b: Unsupervised, Semi-Supervised, and Self-Supervised Learning | c: Supervised, Unsupervised, and Reinforcement Learning | d: Classification, Regression, and Clustering',
  'Correct': 'c'},
 {'MCQ': 'In Supervised Learning, what is the main goal of a Regression task?',
  'Choices': 'a: To predict discrete labels or categories | b: To find patterns in unlabeled data | c: To predict continuous numerical values | d: To learn through trial and error',
  'Correct': 'c'},
 {'MCQ': 'Which of these algorithms is typically used in Unsupervised Learning to group data points into clusters?',
  'Choices': 'a: Linear Regression | b: K-Means clustering | c: Q-Learning | d: Decision Trees',
  'Correct': 'b'},
 {'MCQ': 'Which type of Machine Learning combines a small amount of labeled data with a large amount of unlabeled data, making it useful when labeling is 

In [ ]:
quiz=pd.DataFrame(quiz_table_data)
quiz

,MCQ,Choices,Correct
0,What is the primary goal of Machine Learning?,a: To write explicit rules for every possible ...,b
1,In which type of machine learning do algorithm...,a: Unsupervised Learning | b: Reinforcement Le...,c
2,Which machine learning task involves grouping ...,a: Classification | b: Regression | c: Cluster...,c
3,What problem occurs when a machine learning mo...,a: Underfitting | b: Generalization | c: Overf...,c
4,How does an agent in Reinforcement Learning pr...,a: By being provided with a complete dataset o...,b


In [ ]:
quiz.to_csv("machinelearning.csv",index=False)

In [23]:
import logging
import os
from datetime import datetime

LOG_FILE=f"{datetime.now().strftime('%d_%m_%Y_%H_%M_%S')}.log"
LOG_FILE

'31_12_2025_15_18_19.log'